<a href="https://colab.research.google.com/github/bwrs-1/make_stem/blob/main/Make_stem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# ライブラリのインストール（Colab用）
!apt-get install ffmpeg
!pip install demucs --upgrade

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import os
import subprocess
import shutil
import glob
import time
from datetime import datetime
import traceback  # エラーログを詳細に出力するために追加

# 設定
input_dir = "/content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_BOI_DRUM_KIT/WEST_BULLY_BOI_DRUM_KIT/SAMPLES"
output_base_dir = os.path.join(input_dir, "stem")
converted_dir = "/content/converted_wav"
colab_temp_dir = "demucs_output"
log_path = os.path.join(output_base_dir, "demucs_log.txt")

# フォルダ作成
os.makedirs(output_base_dir, exist_ok=True)
os.makedirs(converted_dir, exist_ok=True)

# ログ書き込み関数
def write_log(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{timestamp}] {message}"
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(line + "\n")
    print(line)

# converted_dir を事前にクリア
def clear_converted_dir():
    deleted_files = 0
    for file in os.listdir(converted_dir):
        file_path = os.path.join(converted_dir, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            deleted_files += 1
    if deleted_files > 0:
        write_log(f"[INIT] converted_dir を事前に {deleted_files} 件クリアしました")
    else:
        write_log("[INIT] converted_dir に残データはありません")

# m4a → wav 変換
def convert_m4a_to_wav(input_path, output_path):
    result = subprocess.run(['ffmpeg', '-y', '-i', input_path, output_path], capture_output=True, text=True)
    if result.returncode != 0:
        write_log(f"[ERROR] m4a → wav 変換失敗: {result.stderr}")
        return False
    return True

# 対象ファイルをすべて取得（input_dir直下のファイルのみ対象）
def get_all_input_files():
    files = []
    for f in os.listdir(input_dir):
        full_path = os.path.join(input_dir, f)
        if os.path.isfile(full_path):
            name, ext = os.path.splitext(f)
            if ext.lower() in ['.wav', '.mp3', '.m4a']:
                files.append(full_path)
    return files

# 単一ファイル処理
def process_single_file(in_path):
    file_start = time.time()
    f = os.path.basename(in_path)
    name, ext = os.path.splitext(f)

    original_path = in_path
    temp_converted_path = None

    # m4a → wav変換
    if ext.lower() == ".m4a":
        temp_converted_path = os.path.join(converted_dir, f"{name}.wav")
        write_log(f"[CONVERT] {f} → {os.path.basename(temp_converted_path)}")
        if not convert_m4a_to_wav(in_path, temp_converted_path):
            return
        in_path = temp_converted_path
    elif ext.lower() == ".wav":
        # wavファイルの場合は変換せずそのまま使用
        write_log(f"[SKIP CONVERT] {f} は既にwavフォーマットのため変換なし")

    try:
        write_log(f"[PROCESS] {f}")
        shutil.rmtree(colab_temp_dir, ignore_errors=True)

        # Demucs v4のモデルを指定して実行
        result = subprocess.run(["python3", "-m", "demucs.separate", in_path, "-o", colab_temp_dir], capture_output=True, text=True)

        if result.returncode != 0:
            write_log(f"[ERROR] {f} 分離に失敗:\n{result.stderr}")
            return

        result_dir = glob.glob(f"{colab_temp_dir}/*/*")[0]
        for stem_file in os.listdir(result_dir):
            stem_type = os.path.splitext(stem_file)[0]
            dest_dir = os.path.join(output_base_dir, stem_type)
            os.makedirs(dest_dir, exist_ok=True)
            new_name = f"{name}_{stem_type}.wav"
            shutil.copy(os.path.join(result_dir, stem_file), os.path.join(dest_dir, new_name))

        # input_dir の元ファイルのみ削除
        if original_path.startswith(input_dir) and os.path.exists(original_path):
            os.remove(original_path)
            write_log(f"[DELETE] input_dir内ファイル削除: {original_path}")

        # converted_dir の一時ファイル削除
        if temp_converted_path and os.path.exists(temp_converted_path):
            os.remove(temp_converted_path)
            write_log(f"[DELETE] converted_dir内変換ファイル削除: {temp_converted_path}")

        elapsed = time.time() - file_start
        write_log(f"[DONE] {f} （{elapsed:.2f} 秒）")

    except Exception as e:
        error_message = f"[ERROR] {f} で例外発生: {str(e)}"
        # スタックトレースをログに追加
        error_traceback = traceback.format_exc()
        write_log(error_message)
        write_log(f"[TRACEBACK] {error_traceback}")

# 事前準備
clear_converted_dir()

# 実行開始
all_files = get_all_input_files()

if not all_files:
    write_log("SAMPLESフォルダに処理対象ファイルが存在しません（サブフォルダ内ファイルは無視）。処理を終了します。")
else:
    write_log(f"\n=== 処理開始: 未処理ファイル数 {len(all_files)} ===")
    write_log("=== 処理対象ファイル一覧 ===")
    for fp in all_files:
        write_log(f" - {fp}")

    for file_path in all_files:
        process_single_file(file_path)

    write_log("\n=== 全処理完了 ===")


[2025-05-11 13:35:49] [INIT] converted_dir に残データはありません
[2025-05-11 13:35:49] 
=== 処理開始: 未処理ファイル数 10 ===
[2025-05-11 13:35:49] === 処理対象ファイル一覧 ===
[2025-05-11 13:35:49]  - /content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_BOI_DRUM_KIT/WEST_BULLY_BOI_DRUM_KIT/SAMPLES/XR3WFACE BANGSY (DELUXE) - 14 The One (prod. by aka3Hundred).wav
[2025-05-11 13:35:49]  - /content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_BOI_DRUM_KIT/WEST_BULLY_BOI_DRUM_KIT/SAMPLES/XR3WFACE_BANGSY_DELUXE_aka3hundred_03_Tf_Is_This_prod_by_aka3hundred.wav
[2025-05-11 13:35:49]  - /content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_BOI_DRUM_KIT/WEST_BULLY_BOI_DRUM_KIT/SAMPLES/XR3WFACE_BANGSY_DELUXE_02_Til_Im_Done_prod_by_aka3Hundred.wav
[2025-05-11 13:35:49]  - /content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_BOI_DRUM_KIT/WEST_BULLY_BOI_DRUM_KIT/SAMPLES/XR3WFACE_BANGSY_DELUXE_aka3hundred_06_Loose_Xrewz_prod_by_aka3hundred.wav
[2025-05-11 13:35:49]  - /content/drive/MyDrive/08_Drum_Kit/01_DRUMS/WEST_BULLY_B